In [1]:
import pickle, pandas, requests, tqdm, time

In [2]:
with open('ptu_spb.pickle', 'rb') as f:
    frame = pickle.load(f)

In [4]:
frame

,first_name,last_name,school,school_type,uid
0,Эдуард,Карлин,109826,1,3826297
1,Виталий,Кузьмин,109826,1,293941751
2,Руслан,Петикян,109826,1,190102307
3,Антон,Павлов,109826,1,48832
4,Александра,Жирова,109826,1,48592866
5,Анастасия,Завьялова,109826,1,108201924
6,Елена,Пузанова,109826,1,24042320
7,Павел,Болеев,109826,1,36571867
8,Оленька,Прокофьева,109826,1,3620120
9,Кристина,Туличева,109826,1,106504528


In [3]:
uids = list(frame['uid'])

def chunks(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

uids = chunks(uids, 24)

In [4]:
api = 'https://api.vk.com/method/'

token = '&access_token=21080f6aa0620f61794f27f0d064c09d19cb44cdd4183f591d5204c74bd5776eca4e41ac94f001f9b9f2a'
api = 'https://api.vk.com/method/'
app_token = '&access_token=a8e5b3eda8e5b3eda8e5b3ed43a8bae255aa8e5a8e5b3edf2e2da74f19ad53d539db181'

In [5]:
data = {
    'sex': [], 'has_photo': [], 'relation': [],
    'political': [], 'religion': [], 'people_main': [], 'life_main': [],
    'smoking': [], 'alchol': [], 'groups': []
}

In [7]:
for uid in tqdm.tqdm_notebook(uids):
    time.sleep(0.3)
    #print(uid)
    sers = requests.get(api + 'execute.getInfoo' + '?idd=' + ','.join(list(map(str, uid))) + token).json()
    
    groups = sers['response'][0]
    #print(groups[1][1]['gid'])
    get = sers['response'][1][0]
    if type(groups) == type(False):
        print(groups)
    for user in groups:
        tmp = []
        if type(user) == type(False):
            data['groups'].append([])
            continue
        for group in user:
            tmp.append(group.setdefault('gid', None))
        data['groups'].append(tmp)
        
    for x in get:
        data['sex'].append(x['sex'])
        data['has_photo'].append(x.setdefault('has_photo', None))
        data['religion'].append(x.setdefault('personal', {}).setdefault('religion', None))
        data['relation'].append(x.setdefault('relation', None))
        data['political'].append(x.setdefault('personal', {}).setdefault('political', None))
        data['people_main'].append(x.setdefault('personal', {}).setdefault('people_main', None))
        data['life_main'].append(x.setdefault('personal', {}).setdefault('life_main', None))
        data['smoking'].append(x.setdefault('personal', {}).setdefault('smoking', None))
        data['alchol'].append(x.setdefault('personal', {}).setdefault('alcohol', None))

### бинаризируем, засовываем в пандас и пикалим

In [8]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [9]:
pandas.DataFrame(data)

,alchol,groups,has_photo,life_main,people_main,political,relation,religion,sex,smoking
0,1.0,[155408923],1,1.0,6.0,4.0,0.0,Православие,2,1.0
1,NaN,"[110192574, 126469688, 144806061, 30192213, 64...",1,NaN,NaN,NaN,6.0,None,2,NaN
2,4.0,"[36775085, 123695926, 31260648, 24881486, 4296...",1,1.0,2.0,NaN,0.0,None,2,2.0
3,NaN,"[38514064, 62684604, 45969278, 81309736, 12736...",1,NaN,NaN,NaN,NaN,None,2,NaN
4,NaN,"[21258485, 28890647, 35026736, 91676820, 11012...",1,NaN,NaN,NaN,0.0,Верю в бога))),1,NaN
5,NaN,"[58737741, 29384413, 34298047, 89718882, 83706...",1,NaN,NaN,NaN,NaN,None,1,NaN
6,NaN,"[49715849, 116801726, 101431498, 74890232, 116...",1,NaN,NaN,NaN,NaN,None,1,NaN
7,NaN,"[35145657, 442, 66544874, 34300760, 62902696, ...",1,NaN,NaN,NaN,2.0,None,2,NaN
8,NaN,"[22222333, 65414135, 36200261, 63648779, 85270...",1,NaN,NaN,NaN,NaN,None,1,NaN
9,NaN,"[152574625, 102943909, 73730654, 35762179, 138...",1,NaN,NaN,NaN,NaN,None,1,NaN
